You have to scrape at least 20000 rows of data. You can scrape more data as well, it’s up to you. more the data better the model
In this section you need to scrape the reviews of different laptops, Phones, Headphones, smart watches, Professional Cameras, Printers, Monitors, Home theater, Router from different e- commerce websites.
Basically, we need these columns-
1) reviews of the product.
2) rating of the product.
You can fetch other data as well, if you think data can be useful or can help in the project. It completely depends on your imagination or assumption.
Hint:
• Try to fetch data from different websites. If data is from different websites, it will help our model to remove the effect of over fitting.
• Try to fetch an equal number of reviews for each rating, for example if you are fetching 10000 reviews then all ratings 1,2,3,4,5 should be 2000. It will balance our data set.
• Convert all the ratings to their round number, as there are only 5 options for rating i.e., 1,2,3,4,5. If a rating is 4.5 convert it 5.

In [ ]:
from urllib.request import urlopen  
from urllib.request import Request  
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image, HTML
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException,WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import urllib.parse
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
driver=webdriver.Chrome()

In [ ]:
#siteinfo={"Amazon":"http://www.amazon.in","Flipkart":"http://www.flipkart.com"}
#product_info={1:'SmartPhone',2:'Laptop',3:'Headphone',4:'smart watch',5:'Professional Camera',6:'Printer',7:'Home theater',8:'Router'}


siteinfo={"Amazon":"https://www.amazon.in"}
for websitename,siteurl in siteinfo.items(): 
    product_info={1:'SmartPhone',2:'Laptop',3:'Headphone',4:'smart watch',5:'Professional Camera',6:'Printer',7:'Home theater',8:'Router'}
    for itemKey,itemValue in product_info.items():
        driver.get(siteurl)
        time.sleep(2)
        secs=20
        search_field_designation=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.ID,'twotabsearchtextbox')))
        search_field_designation=driver.find_element(By.ID,value="twotabsearchtextbox")
        search_field_designation.send_keys(itemValue)
        processorFilter=driver.find_element(By.ID,value="nav-search-submit-button")
        driver.execute_script("arguments[0].click();", processorFilter)
        time.sleep(2)
        urls=[]
        price=[]
        for page in range(20):
            try:
                price_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//a[@class="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]/span[@class="a-price"]/span/span[@class="a-price-whole"]')))
                price_data=driver.find_elements(By.XPATH,'//a[@class="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]/span[@class="a-price"]/span/span[@class="a-price-whole"]')
                product_urls=driver.find_elements(By.XPATH,'//a[@class="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
                for m in range(len(product_urls)):
                    fullurl=product_urls[m].get_attribute('href')
                    if not 'ink' in fullurl.lower():
                        urls.append(fullurl)
                        price.append(price_data[m].text)
                page=page+1
                product_page=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')))
                product_page=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
                product_page.click()
                time.sleep(1)
            except TimeoutException as exception:
                print("Got Exception1")
                continue
        list(dict.fromkeys(urls))
        df_prod_uls=pd.DataFrame({'URL':urls,'Price':price})
        df_prod_uls.to_csv('product_url.csv', mode='a', header=False)
        print(len(urls))
        all_rating_urls=[]
        prod_url=[]
        prod_price=[]
        for p in range(len(urls)):
            driver.get(urls[p])
            time.sleep(1)
            try:             
                full_rating_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//a[@class="a-link-emphasis a-text-bold"]')))
                full_rating_data=driver.find_element(By.XPATH,'//a[@class="a-link-emphasis a-text-bold"]')
                all_rating_url = full_rating_data.get_attribute('href')
                all_rating_urls.append(all_rating_url)
                prod_url.append(urls[p])
                prod_price.append(price[p])
            except TimeoutException as exception:
                print("Got Exception2")
                continue
        list(dict.fromkeys(all_rating_urls))
        df_rr_uls=pd.DataFrame({'rrURL':all_rating_urls,'prod_url':prod_url,'price':prod_price})
        df_rr_uls.to_csv('rr_url.csv', mode='a', header=False)
        print(len(df_rr_uls))
        for n in range(len(all_rating_urls)):
            driver.get(all_rating_urls[n])
            time.sleep(1)
            for y in range(10):
                try:
                    rating_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//div[@class="a-section celwidget"]/div[@class="a-row"]/a[@class="a-link-normal"]/i[contains(@class,"a-icon a-icon-star a-star")]/span[@class="a-icon-alt"]')))
                    rating_data=driver.find_elements(By.XPATH,'//div[@class="a-section celwidget"]/div[@class="a-row"]/a[@class="a-link-normal"]/i[contains(@class,"a-icon a-icon-star a-star")]/span[@class="a-icon-alt"]')
                    review_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//div[@class="a-section celwidget"]/div[@class="a-row"]/a[@class="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"]/span')))
                    review_data=driver.find_elements(By.XPATH,'//div[@class="a-section celwidget"]/div[@class="a-row"]/a[@class="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"]/span')
                    name_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//h1[@class="a-size-large a-text-ellipsis"]')))
                    name_data=driver.find_element(By.XPATH,'//h1[@class="a-size-large a-text-ellipsis"]')
                    reviewCount=len(review_data)
                    final_price_data=price[n]
                    j=0
                    review=[]
                    rating=[]
                    product_name=[]
                    brand_name=[]
                    product_type=[]
                    final_price=[]
                    sitename=[]
                    for i in range(reviewCount):
                        if(review_data[i].text!=''):
                            review.append(review_data[i].text)
                            rating.append(rating_data[j].get_attribute('innerHTML').split(" ")[0])
                            product_name.append(name_data.text)
                            brand_name.append(name_data.text.split(" ")[0])
                            product_type.append(itemValue)
                            sitename.append(websitename)
                            final_price.append(final_price_data)
                            j=j+1
                    df=pd.DataFrame({'WebSite':sitename,'Product_type':product_type,'Brand':brand_name,'Product_name':product_name,'Price':final_price,
                       'Rating':rating,'Review':review})
                    df.to_csv('review_rating_amazon.csv', mode='a', header=False)
                except TimeoutException as exception:
                    print("Got Exception3")
                    continue
                try:
                    time.sleep(3)
                    all_rating_page=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//li[@class="a-last"]')))
                    all_rating_page=driver.find_element(By.XPATH,'//li[@class="a-last"]')
                    driver.execute_script("arguments[0].click();", all_rating_page)
                    time.sleep(3)
                except TimeoutException as exception:
                    print("Got Exception4")
                    break

                    



In [ ]:
#siteinfo={"Amazon":"http://www.amazon.in","Flipkart":"http://www.flipkart.com"}
#product_info={1:'SmartPhone',2:'Laptop',3:'Headphone',4:'smart watch',5:'Professional Camera',6:'Printer',7:'Home theater',8:'Router'}


siteinfo={"Flipkart":"https://www.flipkart.com"}
for websitename,siteurl in siteinfo.items(): 
    product_info={1:'SmartPhone',2:'Laptop',3:'Headphone',4:'smart watch',5:'Professional Camera',6:'Printer',7:'Home theater',8:'Router'}
    for itemKey,itemValue in product_info.items():
        driver.get(siteurl)
        time.sleep(2)
        secs=10
        try:
            driver.find_element(By.XPATH,value="/html/body/div[2]/div/div/button").click()
        except NoSuchElementException:
            z=1        
        search_field_designation=driver.find_element(By.CLASS_NAME,value='_3704LK')
        search_field_designation.send_keys(itemValue)
        time.sleep(2)
        processorFilter = driver.find_element(By.CLASS_NAME,value='L0Z3Pu')
        driver.execute_script("arguments[0].click();", processorFilter)
        time.sleep(2)
        urls=[]
        price=[]
        for page in range(19):
            product_urls=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//a[@class="s1Q9rs"]')))
            product_urls=driver.find_elements(By.XPATH,'//a[@class="s1Q9rs"]')
            price_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//div[@class="_30jeq3"]')))
            price_data=driver.find_elements(By.XPATH,'//div[@class="_30jeq3"]')
            for m in range(len(product_urls)):
                if 'home theatre' in product_urls[m].text.lower():
                    fullurl=product_urls[m].get_attribute('href')
                    urls.append(fullurl)
                    price.append(price_data[m].text)
            page=page+1
            nextTag=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//a[@class="_1LKTO3"]')))
            nextTag=driver.find_elements(By.XPATH,'//a[@class="_1LKTO3"]')
            tagcount=len(nextTag)
            nextTag[tagcount-1].click()
            time.sleep(3)
        list(dict.fromkeys(urls)) 
        print(len(urls))
        for n in range(len(urls)):
            driver.get(urls[n])  
            time.sleep(4)
            try:
                rating_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//div[starts-with(@class, "_3LWZlK") and contains(@class,"_1BLPMq")]')))
                rating_data=driver.find_elements(By.XPATH,'//div[starts-with(@class, "_3LWZlK") and contains(@class,"_1BLPMq")]')
                review_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//p[@class="_2-N8zT"]')))
                review_data=driver.find_elements(By.XPATH,'//p[@class="_2-N8zT"]')
                name_data=WebDriverWait(driver, secs).until(EC.presence_of_element_located((By.XPATH,'//span[@class="B_NuCI"]')))
                name_data=driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')
                reviewCount=len(review_data)
                final_price_data=price[n]
                review=[]
                rating=[]
                product_name=[]
                brand_name=[]
                product_type=[]
                final_price=[]
                sitename=[]
                print(urls[n])
                print(reviewCount)
                print(len(rating_data))
                for i in range(reviewCount):
                    if(review_data[i].text!=''):
                        review.append(review_data[i].text)
                        rating.append(rating_data[i].text)
                        product_name.append(name_data.text)
                        brand_name.append(name_data.text.split(" ")[0])
                        product_type.append(itemValue)
                        sitename.append(websitename)
                        final_price.append(final_price_data)
                df=pd.DataFrame({'WebSite':sitename,'Product_type':product_type,'Brand':brand_name,'Product_name':product_name,'Price':final_price,
                   'Rating':rating,'Review':review})
                df
                df.to_csv('review_rating_flipkart.csv', mode='a', header=False)
            except TimeoutException as exception:
                continue

